# Laboratorio 2: Semántica Vectorial.


### Cuerpo Docente

- Profesores: [Andrés Abeliuk](https://aabeliuk.github.io/), [Fabián Villena](https://villena.cl/).
- Profesor Auxiliar: [Martín Paredes](https://github.com/MartinParedesR/)

Estas son las librerías permitidas. Si quieren utilizar alguna librería adicional, pueden realizar la consulta a través de foro.

In [36]:
import re
import numpy as np
import pandas as pd


**Ejercicio 1 - *Bag of Words* 🐶🐈.**



Considere el siguiente corpus, donde cada elemento del arreglo representa un documento:

**disclaimer: El orden de los resultados pueden variar**

In [37]:
d0 = 'El perro se come la comida y después se duerme'
d1 = 'El perro se despierta y después empieza a ladrar'
d2 = 'El perro ladra y después se come la comida'
d3 = 'El gato se come la comida y después se duerme'
d4 = 'El gato se despierta y después empieza a maullar'
d5 = 'El gato maulla y después se come la comida'
dataset = [d0, d1, d2, d3, d4, d5]

El objetivo de este ejercicio es determinar cuáles de  los documentos entregados son los más similares entre sí. Para ello utilizaremos la técnica TF-IDF.

Como los algoritmos de Machine Learning no comprenden el texto en lenguaje natural, estos documentos deben ser convertidos a vectores numéricos. La representación más simple vista en clases es el **Bag of Words**, método mediante el cuál se cuentan las apariciones de cada palabra en cada uno de los documentos entregados.

Implemente la función **`bag_of_words()`**, que reciba como input un arreglo de documentos y devuelva un pandas dataframe con la representación Bag of Words de los documentos entregados. En esta representación las columnas son el vocabulario y las filas representa las apariciones de cada una de las palabras en los documentos. En otras palabras, cada fila representa el bow de un determinado documento.


Por ejemplo para el siguiente dataset:

```
dataset = ['El perro ladra', 'El perro come']
```

Debiese entregarnos lo siguiente:


|   | el | perro | ladra | come |
|---|----|-------|------|-------|
| 0 | 1  | 1     | 1    | 0     |
| 1 | 1  | 1     | 0    | 1     |



In [38]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Podría serle útil primero definir una funcion `get_vocab` y `tokenize`.

In [39]:
#Normaliza una cadena de texto y obtiene sus token
def tokenize(text):
  text = re.sub(r'[^A-Za-zñáéíóú]', ' ', text)
  return text.split()

In [40]:
tokenize(d0)

['El', 'perro', 'se', 'come', 'la', 'comida', 'y', 'después', 'se', 'duerme']

In [41]:
def get_vocab(corpus):
    vocabulary = []
    for document in corpus:
        tokens = tokenize(document)
        for token in tokens:
            if token not in vocabulary:
                vocabulary.append(token)
    return vocabulary

In [42]:
def bag_of_words(corpus):
    vocabulary = get_vocab(corpus)
    df = pd.DataFrame(columns=vocabulary) #Data frame con vocabulario como columnas

    idx = 0
    for document in corpus:
        doct_dict = {}
        #Inicializar vocabulario
        for word in vocabulary:
            doct_dict[word] = 0
        #Agregar ocurrencias
        for word in tokenize(document):
            doct_dict[word] += 1
        #Índice por palabra
        df.loc["d"+str(idx)] = doct_dict

        idx += 1

    return df

***Test:***

In [43]:
dataset_bow = bag_of_words(dataset)
dataset_bow

,El,perro,se,come,la,comida,y,después,duerme,despierta,empieza,a,ladrar,ladra,gato,maullar,maulla
d0,1,1,2,1,1,1,1,1,1,0,0,0,0,0,0,0,0
d1,1,1,1,0,0,0,1,1,0,1,1,1,1,0,0,0,0
d2,1,1,1,1,1,1,1,1,0,0,0,0,0,1,0,0,0
d3,1,0,2,1,1,1,1,1,1,0,0,0,0,0,1,0,0
d4,1,0,1,0,0,0,1,1,0,1,1,1,0,0,1,1,0
d5,1,0,1,1,1,1,1,1,0,0,0,0,0,0,1,0,1


``
``

***Resultado esperado***:

|    | El | perro | se | come | la | comida | y | después | duerme | despierta | empieza | a | ladrar | ladra | gato | maullar | maulla |
|----|---:|------:|---:|-----:|---:|-------:|--:|--------:|-------:|----------:|--------:|--:|-------:|------:|-----:|--------:|-------:|
| d0 |  1 |     1 |  2 |    1 |  1 |      1 | 1 |       1 |      1 |         0 |       0 | 0 |      0 |     0 |    0 |       0 |      0 |
| d1 |  1 |     1 |  1 |    0 |  0 |      0 | 1 |       1 |      0 |         1 |       1 | 1 |      1 |     0 |    0 |       0 |      0 |
| d2 |  1 |     1 |  1 |    1 |  1 |      1 | 1 |       1 |      0 |         0 |       0 | 0 |      0 |     1 |    0 |       0 |      0 |
| d3 |  1 |     0 |  2 |    1 |  1 |      1 | 1 |       1 |      1 |         0 |       0 | 0 |      0 |     0 |    1 |       0 |      0 |
| d4 |  1 |     0 |  1 |    0 |  0 |      0 | 1 |       1 |      0 |         1 |       1 | 1 |      0 |     0 |    1 |       1 |      0 |
| d5 |  1 |     0 |  1 |    1 |  1 |      1 | 1 |       1 |      0 |         0 |       0 | 0 |      0 |     0 |    1 |       0 |      1 |

``
``

` `  
` `

**Ejercicio 2 - *Calcular TF* (0.5 puntos):** Ahora debemos usar el dataframe del ejercicio anterior para calcular la matriz de TF normalizada por la máxima frecuencia ${max_i({\text{tf}_{i,j})}}$, donde
i corresponde al índice de las filas (bow) y j al de las columnas (palabras). Es decir, dividir cada bow en la cantidad de veces de la palabra que aparezca más veces en ese vector.


$$\text{nft}_{i,j} = \frac{\text{tf}_{i,j}}{max_i({\text{tf}_{i,j})}}$$

In [55]:
def calc_tf(dataset_bow):
    df = pd.DataFrame(np.zeros((dataset_bow.shape[0], dataset_bow.shape[1])))
    df = df.set_axis(dataset_bow.columns, axis='columns')
    df = df.set_axis(dataset_bow.index, axis='index')
    for i in dataset_bow.index:
      max_freq = dataset_bow.loc[i].max()
      for j in dataset_bow.columns:
        df.loc[i,j] = dataset_bow.loc[i,j] / max_freq

    return df

***Test:***

In [56]:
tf = calc_tf(dataset_bow)
tf

,El,perro,se,come,la,comida,y,después,duerme,despierta,empieza,a,ladrar,ladra,gato,maullar,maulla
d0,0.5,0.5,1.0,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d1,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
d2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
d3,0.5,0.0,1.0,0.5,0.5,0.5,0.5,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0
d4,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
d5,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


``
``

***Resultado esperado***:

|    |  El | perro |  se | come |  la | comida |   y | después | duerme | despierta | empieza |   a | ladrar | ladra | gato | maullar | maulla |
|----|----:|------:|----:|-----:|----:|-------:|----:|--------:|-------:|----------:|--------:|----:|-------:|------:|-----:|--------:|-------:|
| d0 | 0.5 |   0.5 | 1.0 |  0.5 | 0.5 |    0.5 | 0.5 |     0.5 |    0.5 |       0.0 |     0.0 | 0.0 |    0.0 |   0.0 |  0.0 |     0.0 |    0.0 |
| d1 | 1.0 |   1.0 | 1.0 |  0.0 | 0.0 |    0.0 | 1.0 |     1.0 |    0.0 |       1.0 |     1.0 | 1.0 |    1.0 |   0.0 |  0.0 |     0.0 |    0.0 |
| d2 | 1.0 |   1.0 | 1.0 |  1.0 | 1.0 |    1.0 | 1.0 |     1.0 |    0.0 |       0.0 |     0.0 | 0.0 |    0.0 |   1.0 |  0.0 |     0.0 |    0.0 |
| d3 | 0.5 |   0.0 | 1.0 |  0.5 | 0.5 |    0.5 | 0.5 |     0.5 |    0.5 |       0.0 |     0.0 | 0.0 |    0.0 |   0.0 |  0.5 |     0.0 |    0.0 |
| d4 | 1.0 |   0.0 | 1.0 |  0.0 | 0.0 |    0.0 | 1.0 |     1.0 |    0.0 |       1.0 |     1.0 | 1.0 |    0.0 |   0.0 |  1.0 |     1.0 |    0.0 |
| d5 | 1.0 |   0.0 | 1.0 |  1.0 | 1.0 |    1.0 | 1.0 |     1.0 |    0.0 |       0.0 |     0.0 | 0.0 |    0.0 |   0.0 |  1.0 |     0.0 |    1.0 |

``
``

` `  
` `

**Ejercicio 3 - *Calcular IDF*:**


Implementar `calc_idf(dataset_bow)`. Este debe retornar un diccionario en donde las llaves sean las palabras y los valores sean el calculo de cada idf por palabra.

Recordar que $idf_{t_i} = log_{10}\frac{N}{n_i}$ con $N = $ número de documentos y $n_i = $ Número de documentos que contienen la palabra $t_i$

In [57]:
def calc_idf(dataset_bow):
    N=dataset_bow.shape[0]
    claves = dataset_bow.columns.to_list()
    valores=[]
    for i in claves:
      valores.append(np.log10(N/(dataset_bow[i]!=0).sum()))

    diccionario = {claves[i]: valores[i] for i in range(len(claves))}

    return(diccionario)

***Test:***

In [58]:
idf = calc_idf(dataset_bow)
idf

{'El': np.float64(0.0),
 'perro': np.float64(0.3010299956639812),
 'se': np.float64(0.0),
 'come': np.float64(0.17609125905568124),
 'la': np.float64(0.17609125905568124),
 'comida': np.float64(0.17609125905568124),
 'y': np.float64(0.0),
 'después': np.float64(0.0),
 'duerme': np.float64(0.47712125471966244),
 'despierta': np.float64(0.47712125471966244),
 'empieza': np.float64(0.47712125471966244),
 'a': np.float64(0.47712125471966244),
 'ladrar': np.float64(0.7781512503836436),
 'ladra': np.float64(0.7781512503836436),
 'gato': np.float64(0.3010299956639812),
 'maullar': np.float64(0.7781512503836436),
 'maulla': np.float64(0.7781512503836436)}

***Resultado esperado***:

```python
{'El': np.float64(0.0),
 'perro': np.float64(0.3010299956639812),
 'se': np.float64(0.0),
 'come': np.float64(0.17609125905568124),
 'la': np.float64(0.17609125905568124),
 'comida': np.float64(0.17609125905568124),
 'y': np.float64(0.0),
 'después': np.float64(0.0),
 'duerme': np.float64(0.47712125471966244),
 'despierta': np.float64(0.47712125471966244),
 'empieza': np.float64(0.47712125471966244),
 'a': np.float64(0.47712125471966244),
 'ladrar': np.float64(0.7781512503836436),
 'ladra': np.float64(0.7781512503836436),
 'gato': np.float64(0.3010299956639812),
 'maullar': np.float64(0.7781512503836436),
 'maulla': np.float64(0.7781512503836436)}
```

Puede notar el bajo puntaje otorgado a las palabras que más se repiten! 😮

**Ejercicio 4 - *Calcular TF-IDF***


Programe la función `calc_tf_idf(tf, idf)` que entrega el dataframe TF-IDF asociado al dataset que estamos analizando.

In [59]:
def calc_tf_idf(tf, idf):
    df=pd.DataFrame(np.zeros((tf.shape[0], tf.shape[1])))
    df = df.set_axis(tf.columns, axis='columns')
    df = df.set_axis(tf.index, axis='index')
    for i in tf.columns:
      for j in tf.index:
        df.loc[j,i] = tf.loc[j,i]*idf[i]

    return df

***Test.***

In [60]:
tf_idf = calc_tf_idf(tf, idf)
tf_idf

,El,perro,se,come,la,comida,y,después,duerme,despierta,empieza,a,ladrar,ladra,gato,maullar,maulla
d0,0.0,0.150515,0.0,0.088046,0.088046,0.088046,0.0,0.0,0.238561,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
d1,0.0,0.301030,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.477121,0.477121,0.477121,0.778151,0.000000,0.000000,0.000000,0.000000
d2,0.0,0.301030,0.0,0.176091,0.176091,0.176091,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.778151,0.000000,0.000000,0.000000
d3,0.0,0.000000,0.0,0.088046,0.088046,0.088046,0.0,0.0,0.238561,0.000000,0.000000,0.000000,0.000000,0.000000,0.150515,0.000000,0.000000
d4,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.477121,0.477121,0.477121,0.000000,0.000000,0.301030,0.778151,0.000000
d5,0.0,0.000000,0.0,0.176091,0.176091,0.176091,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.301030,0.000000,0.778151


``
``

***Resultado esperado***:

|    |  El |    perro |  se |     come |       la |   comida |   y | después |   duerme | despierta |  empieza |        a |   ladrar |    ladra |     gato |  maullar |   maulla |
|----|----:|---------:|----:|---------:|---------:|---------:|----:|--------:|---------:|----------:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|
| d0 | 0.0 | 0.150515 | 0.0 | 0.088046 | 0.088046 | 0.088046 | 0.0 |     0.0 | 0.238561 |  0.000000 | 0.000000 | 0.000000 | 0.000000 | 0.000000 | 0.000000 | 0.000000 | 0.000000 |
| d1 | 0.0 | 0.301030 | 0.0 | 0.000000 | 0.000000 | 0.000000 | 0.0 |     0.0 | 0.000000 |  0.477121 | 0.477121 | 0.477121 | 0.778151 | 0.000000 | 0.000000 | 0.000000 | 0.000000 |
| d2 | 0.0 | 0.301030 | 0.0 | 0.176091 | 0.176091 | 0.176091 | 0.0 |     0.0 | 0.000000 |  0.000000 | 0.000000 | 0.000000 | 0.000000 | 0.778151 | 0.000000 | 0.000000 | 0.000000 |
| d3 | 0.0 | 0.000000 | 0.0 | 0.088046 | 0.088046 | 0.088046 | 0.0 |     0.0 | 0.238561 |  0.000000 | 0.000000 | 0.000000 | 0.000000 | 0.000000 | 0.150515 | 0.000000 | 0.000000 |
| d4 | 0.0 | 0.000000 | 0.0 | 0.000000 | 0.000000 | 0.000000 | 0.0 |     0.0 | 0.000000 |  0.477121 | 0.477121 | 0.477121 | 0.000000 | 0.000000 | 0.301030 | 0.778151 | 0.000000 |
| d5 | 0.0 | 0.000000 | 0.0 | 0.176091 | 0.176091 | 0.176091 | 0.0 |     0.0 | 0.000000 |  0.000000 | 0.000000 | 0.000000 | 0.000000 | 0.000000 | 0.301030 | 0.000000 | 0.778151 |


``
``

***Ejercicio 5: Concluir con similitud coseno***

Ahora que tenemos el dataframe de TF-IDF, nos queda calcular la similitud coseno entre todos los vectores. Notar que la matriz resultante será una matriz simétrica. Implemente la función *cosine_similarity(v1, v2)* que recibe dos vectores (v1 y v2) y calcula la similitud coseno entre ambos vectores. Concluya cuáles son los dos documentos más similares.

In [61]:
def cosine_similarity(v1, v2):

    sum_producto_punto = 0
    for i in range(len(v1)):
        sum_producto_punto += v1[i] * v2[i]

    sum_v1 = 0
    for i in range(len(v1)):
        sum_v1 += v1[i] ** 2
    raiz_v1 = np.sqrt(sum_v1)
    sum_v2 = 0
    for i in range(len(v2)):
        sum_v2 += v2[i] ** 2
    raiz_v2 = np.sqrt(sum_v2)
    producto_denominador = raiz_v1 * raiz_v2

    resultado = sum_producto_punto / producto_denominador

    return resultado

similarity_matrix = np.zeros((6,6))
for i, v1 in enumerate(tf_idf.index.values):
  for j, v2 in enumerate(tf_idf.index.values):
      similarity = cosine_similarity(tf_idf.loc[v1].values, tf_idf.loc[v2].values)
      similarity_matrix[i][j] = similarity

for i in range(6):
  mask = [k != i for k in range(6)]
  j = np.argmax(similarity_matrix[i][mask])

  print(dataset[i])
  print("> Mas similar:", np.array(dataset)[mask][j])
  print("> Similitud:", similarity_matrix[i][mask][j], "\n")

El perro se come la comida y después se duerme
> Mas similar: El gato se come la comida y después se duerme
> Similitud: 0.7796701424731538 

El perro se despierta y después empieza a ladrar
> Mas similar: El gato se despierta y después empieza a maullar
> Similitud: 0.49521259700657505 

El perro ladra y después se come la comida
> Mas similar: El perro se come la comida y después se duerme
> Similitud: 0.3223435954694041 

El gato se come la comida y después se duerme
> Mas similar: El perro se come la comida y después se duerme
> Similitud: 0.7796701424731538 

El gato se despierta y después empieza a maullar
> Mas similar: El perro se despierta y después empieza a ladrar
> Similitud: 0.49521259700657505 

El gato maulla y después se come la comida
> Mas similar: El gato se come la comida y después se duerme
> Similitud: 0.3223435954694041 



***Resultado esperado***:
```
El perro se come la comida y después se duerme
> Mas similar: El gato se come la comida y después se duerme
> Similitud: 0.7796701424731538

El perro se despierta y después empieza a ladrar
> Mas similar: El gato se despierta y después empieza a maullar
> Similitud: 0.49521259700657505

El perro ladra y después se come la comida
> Mas similar: El perro se come la comida y después se duerme
> Similitud: 0.3223435954694041

El gato se come la comida y después se duerme
> Mas similar: El perro se come la comida y después se duerme
> Similitud: 0.7796701424731538

El gato se despierta y después empieza a maullar
> Mas similar: El perro se despierta y después empieza a ladrar
> Similitud: 0.49521259700657505

El gato maulla y después se come la comida
> Mas similar: El gato se come la comida y después se duerme
> Similitud: 0.3223435954694041
```

![gato](https://live.staticflickr.com/4652/38904147065_0b6c446945_b.jpg)

**Cualquier recomendación que nos quisieran dar para un futuro lab es bienvenida!**